<a href="https://colab.research.google.com/github/wakamatsuikuma/scratch-deeplearning/blob/main/Ch5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# coding: utf-8

import sys, os
ｓys.path.append("/content/drive/MyDrive/ゼロからつくるディープラーニング")
ｓys.path.append("/content/drive/MyDrive/ゼロからつくるディープラーニング/dataset")
ｓys.path.append("/content/drive/MyDrive/ゼロからつくるディープラーニング/common")
ｓys.path.append("/content/drive/MyDrive/ゼロからつくるディープラーニング/Ch4")

import numpy as np
from mnist import load_mnist
from PIL import Image
from IPython.display import display
import pickle
from functions import sigmoid, softmax
from two_layer_net import TwoLayerNet


# グーグルコラボなので画像表示はこれ使う
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    display(pil_img)

# ファイルをアップロード
from google.colab import files
uploaded = files.upload()

In [ ]:
"""
乗算レイヤの実装
 順伝播と逆伝播のメソッドを実装
"""

class MulLayer:
  # ｘ、ｙの初期化。順伝播時の入力値を保持するため。
  def __init__(self):
    self.x = None
    self.y = None
  
  def forward(self, x, y):
    self.x = x
    self.y = y
    out = self.x * self.y

    return out
  
  # 微分(dout)に対して"ひっくり返した値"をかけて流す
  def backward(self, dout):
    dx = dout * self.y
    dy = dout * self.x

    return dx, dy

    

In [ ]:
"""
りんご2個の買い物　を実装
"""

# 入力は単価、個数、消費税
apple = 100
apple_num = 2
tax = 1.1


# 各レイヤのインスタンス生成
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()


# 価格(順伝播)の計算
apple_price = mul_apple_layer.forward(apple, apple_num)
total_price = mul_tax_layer.forward(apple_price, tax)

print(apple_price, total_price)


# 各レイヤの微分(呼び出す順番は順伝播の逆)
dtotal_price = 1

dapple_price, dtax = mul_tax_layer.backward(dtotal_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

200 220.00000000000003
2.2 110.00000000000001 200


In [ ]:
"""
加算レイヤの実装
"""

class AddLayer:
  # インスタンス生成。保持する必要のある値は無い。
  def __init__(self):
    pass
  
  # 順伝播
  def forward(self, x, y):
    out = x + y

    return out

  # 逆伝播
  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1
    
    return dx, dy


In [ ]:
"""
りんご2個とみかん3個の買い物
"""
# 入力値
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 各レイヤのインスタンス生成
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()


# 価格を計算 = 順伝播
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
add_price = add_apple_orange_layer.forward(apple_price, orange_price)
total_price = mul_tax_layer.forward(add_price, tax)

print(apple_price, orange_price, add_price, total_price)


# 各入力の微分を計算 = 逆伝播
dtotal_price = 1

dadd_price, dtax = mul_tax_layer.backward(dtotal_price)
dapple_price, dorange_price = add_apple_orange_layer.backward(dadd_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("total_price:", int(total_price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)


200 450 650 715.0000000000001
total_price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


In [ ]:
"""
sigmoidレイヤの実装
"""
import numpy as np

class Sigmoid:
  
  def __init__(self):
    self.out = None

  def forward(self, x):
    out = 1 / (1 + np.exp(-x))
    self.out = out # 逆伝播時に使うので、インスタンス変数に保持

    return out
  
  def backward(self, dout):
    dx = dout * sef.out * (1.0 - self.out)

    return dx


In [ ]:
"""
Affine変換の実装
"""
import numpy as np

class Affine:
  def __init__(self, W, b):
    self.W = W
    self.b = b
    self.x = None
    self.original_x_shape = None # テンソル対応のため形状を保持

    self.dw = None
    self.db = None

  def forward(self, x):
    self.original_x_shape = x.shape
    x = x.reshape(x.shape[0], -1)
    self.x = x
    out = np.dot(self.x, self.w) + self.b
    
    return out


  def backward(self, dout):
    dx = np.dot(dout, self.W.T)
    dx = dx.reshape(*self.original_x_shape) # 入力データの形状に戻す(テンソル対応)

    self.dW = np.dot(self.x.T, dout)
    self.db = np.sum(dout, axis=0)

    retutn dx
    